# Langchain agent - Perform math on search results
#### Set path context

In [1]:
## Set path context
%load_ext autoreload
%autoreload 2
import os
import sys
module_path = os.path.abspath(os.path.join('../')) # or the path to your source code
sys.path.insert(0, module_path)
print(sys.path)

['/workspaces/genai-ws-main/genai_workshop', '/workspaces/genai-ws-main/genai_workshop/example_notebook', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages']


### Import dependencies

In [2]:
!pip install wikipedia numexpr duckduckgo-search

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 KB 6.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 KB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 39.7 MB/s eta 0:00:0000:0100:01
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11680 sha256=f0a21c8b4d354810b43ee6d7f2c77577625630ec9e9e874d3f1ea0226962d786
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.12
    Uninstalling orjson-3.9.12:
      Successfully uninstalled orjson-3.9.12


In [3]:

from langchain.llms import OpenAI
from langchain.agents import Tool, initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.tools import DuckDuckGoSearchRun
from langchain.utilities import WikipediaAPIWrapper
from langchain import OpenAI, LLMMathChain
from langchain.chains import LLMChain
from langchain.agents.agent_types import AgentType
from prompt_utils.openai_utils import get_openai_chat_model

### Setting up the internet search Tool

In [4]:
search = DuckDuckGoSearchRun()


### LLM math chain

In [5]:
from langchain import LLMMathChain
llm_model = ChatOpenAI(model_name="gpt-4", temperature=0)
llm_math_chain = LLMMathChain.from_llm(llm=llm_model, verbose=True)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


### Initialize tools and agents

In [6]:

# Search Tool
search_tool = Tool(
    name="Search",
    func=search.run,
    description="A useful tool for searching the Internet to find information on world events, issues, dates, years, etc. Worth using for general topics. Use precise questions.",
)


# Math tool
math_tool = Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only input math expressions.",
    )

agent = initialize_agent(
    tools = [search_tool,math_tool],
    llm=llm_model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,  
)

### Agent template

In [7]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: A useful tool for searching the Internet to find information on world events, issues, dates, years, etc. Worth using for general topics. Use precise questions.
Calculator: Useful for when you need to answer questions about math. This tool is only for math questions and nothing else. Only input math expressions.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


### Run query against agent

In [8]:

#query = "Determine the duration of a journey from India to the United States when traveling at the speed of sound"
query = "Find out the flight distance between India and US in kilometers. Also Find out the speed of sound in kilometers per hour.  Calculate the time in hours to travel from India to US with speed of sound."
agent.run(query)



> Entering new AgentExecutor chain...


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your-api*****here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}